In [15]:
from  datetime import datetime, timedelta
# import gc
import numpy as np, pandas as pd
import time	


In [16]:
PRICE_DTYPES = {"store_id": "category", "item_id": "category", "wm_yr_wk": "int16","sell_price":"float32" }
prices = pd.read_csv("../datasets/sell_prices.csv", dtype = PRICE_DTYPES)

for col, col_dtype in PRICE_DTYPES.items():
    if col_dtype == "category":
        prices[col] = prices[col].cat.codes.astype("int16")  # 类似于labelencoder，将store_id和item_id数字化表示
        prices[col] -= prices[col].min()  



In [17]:
CAL_DTYPES={"event_name_1": "category", "event_name_2": "category", "event_type_1": "category", 
         "event_type_2": "category", "weekday": "category", 'wm_yr_wk': 'int16', "wday": "int16",
        "month": "int16", "year": "int16", "snap_CA": "float32", 'snap_TX': 'float32', 'snap_WI': 'float32' }
cal = pd.read_csv("../datasets/calendar.csv", dtype = CAL_DTYPES)

cal["date"] = pd.to_datetime(cal["date"])  #  转化为datetime类型
for col, col_dtype in CAL_DTYPES.items():
    if col_dtype == "category":
        cal[col] = cal[col].cat.codes.astype("int16")  # 将event_name_1/event_type_1/event_name_2/event_type_2weekday数字化表示
        cal[col] -= cal[col].min()  # NaN会转换为-1，该代表将NaN归为0，其他从1开始

In [18]:
h = 28 
# max_lags = 57
last_day = 1913  # 历史的最后一天
fday = datetime(2016,4, 25)  # 预测的第一天
fday
# first_day = 1750
first_day = 1850
# demo_rows = 10000 # should be all
demo_rows = None # should be all

In [19]:
numcols = [f"d_{day}" for day in range(first_day, last_day + 1)]  # 选取历史数据，从first_day到tr_last（包含这一天）d_1200,d_1201
catcols = ['id', 'item_id', 'dept_id','store_id', 'cat_id', 'state_id']
dtype = {numcol: "float32" for numcol in numcols}   # d_1200为float32
dtype.update({col: "category" for col in catcols if col != "id"})  # 除id外，都为category类型

train_data = pd.read_csv("../datasets/sales_train_validation.csv", usecols = catcols + numcols, dtype = dtype,
                         nrows=demo_rows )  # 取catcols和numcols的列(只取了需要的历史天数销量数据)
print(train_data.shape)
# print(train_data.info())
train_data.head(3)

(30490, 70)


,id,item_id,dept_id,cat_id,store_id,state_id,d_1850,d_1851,d_1852,d_1853,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0.0,4.0,0.0,1.0,...,1.0,3.0,0.0,1.0,1.0,1.0,3.0,0.0,1.0,1.0
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,1.0,0.0,0.0,0.0,...,2.0,1.0,2.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0


In [20]:
for col in catcols:
    if col != "id":
        train_data[col] = train_data[col].cat.codes.astype("int16")  # 数字化
        train_data[col] -= train_data[col].min()

    
for day in range(last_day + 1, last_day + 28 +1):  # 遍历预测的每一天
    train_data[f"d_{day}"] = np.nan  # 先填充为NaN

train_data = pd.melt(train_data,
              id_vars = catcols,
              value_vars = [col for col in train_data.columns if col.startswith("d_")],
              var_name = "d",
              value_name = "sales")

In [21]:
train_data = train_data.merge(cal, on= "d", copy = False)
train_data = train_data.merge(prices, on = ["store_id", "item_id", "wm_yr_wk"], copy = False)

In [22]:
train_data["d"]=train_data["d"].apply(lambda x: int(x.split("_")[1])) # d_1 to 1


In [23]:
# train_data.to_csv("train_data.csv",index=False)
train_data.to_csv("train_data.csv.gz",compression='gzip',index=False)